In [ ]:
!pip install openai==0.28
!pip install pandas as pd

ERROR: Could not find a version that satisfies the requirement as (from versions: none)
ERROR: No matching distribution found for as


# Using gpt-3.5 turbo gives Acc -> 72

In [ ]:
import openai
import pandas as pd

# Initialize OpenAI API key
openai.api_key = 'XXXXX'

# Load your dataset (ensure the dataset has columns 'news' and 'headline')
data = pd.read_csv('unlabelled_test_set_t3.csv')

import re

# Function to query GPT-4 API with the context (full news content and headline)
def get_response(news_content, headline):
    messages = [
        {
            "role": "system",
            "content": "You are an AI assistant that helps fill in the missing numbers in news headlines based on the content of the news."
        },
        {
            "role": "user",
            "content": f"Based on the following news: '{news_content}', fill in the missing number in this headline: '{headline}'"
        }
    ]
    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",  # You can switch to 'gpt-4' if available
            messages=messages,
            max_tokens=100,  # Reduce token limit for concise responses
            temperature=0.3
        )
        result = response['choices'][0]['message']['content'].strip()
        # Extract numeric value using regex
        match = re.search(r'\d+', result)
        if match:
            return match.group()
        else:
            return "No number found in response."
    except Exception as e:
        print(f"Error getting response from OpenAI: {e}")
        return "An error occurred while processing your query."

# Apply the prediction function to each row
def predict_missing_value(row):
    news_content = row['news']   # Full news content from the 'news' column
    headline = row['headline']   # Headline with a missing number
    return get_response(news_content, headline)

# Create a new column 'predicted_value' in the DataFrame by applying the function
data['predicted_value'] = data.apply(predict_missing_value, axis=1)

# Save the output to a new CSV file
data.to_csv('last/lastpredicted_news_data.csv', index=False)

In [ ]:
predicted = pd.read_csv('last/lastpredicted_news_data.csv')

In [ ]:
predicted

,ID,news,headline,predicted_value
0,TEST_1,"(Dec 6, 2019 2:50 PM) Elizabeth Warren is in e...",Warren Is First of ____-Something Candidates O...,70
1,TEST_2,"(Sep 20, 2019 1:52 AM CDT) Gun-maker Colt is s...",Colt Suspends Production of AR-____ Rifles for...,15
2,TEST_3,"(Oct 19, 2020 3:00 AM CDT) David Perdue's perc...","He Mocked Kamala's Name, Scored $____M... for ...",1
3,TEST_4,"(Jul 1, 2020 8:41 AM CDT) You're not supposed ...","Here Comes the Sunâ€”____ Years of It, in One ...",10
4,TEST_5,"(Aug 28, 2020 2:33 PM CDT) Health officials in...",____ Test Positive for COVID at RNC,No number found in response.
...,...,...,...,...
704,TEST_705,"(Oct 23, 2010 1:30 PM CDT) Hollywood's million...",Liam Neeson Nabs Mel's Spot in Hangover ____,2
705,TEST_706,"(May 19, 2009 12:25 PM CDT) Los Angeles police...",Dolla Shooting Suspect Held on $____M Bail,1
706,TEST_707,"(Apr 14, 2012 4:29 PM CDT) Purdue University e...",Rube Goldberg Machine Pops Balloon in ____ Steps,300
707,TEST_708,"(Apr 14, 2012 2:15 PM CDT) Egypt's election co...",Egypt Bars ____ Hopefuls From Presidential Ele...,10


# Using gpt-4 Acc -> 85

In [ ]:
import openai
import pandas as pd

# Initialize OpenAI API key
openai.api_key = 'XXXX'  # Replace with your actual OpenAI API key

# Load your dataset (ensure the dataset has columns 'news' and 'headline')
data = pd.read_csv('unlabelled_test_set_t3.csv')

import re

# Function to query GPT-4 API with the context (full news content and headline), including examples
def get_response(news_content, headline):
    messages = [
        {
            "role": "system",
            "content": "You are an AI assistant that helps fill in the missing numbers in news headlines based on the content of the news. Always return the number in numeric form, even if the number is initially written in words. For example, if the number is 'four', return '4'."
        },
        {
            "role": "user",
            "content": f"Based on the following news: '{news_content}', fill in the missing number in this headline: '{headline}'"
        }
    ]
    try:
        response = openai.ChatCompletion.create(
            model="gpt-4",  # Switch to GPT-4
            messages=messages,
            max_tokens=4000,  # Limit for concise responses
            temperature=0.3
        )
        result = response['choices'][0]['message']['content'].strip()
        # Extract numeric value using regex
        match = re.search(r'\d+', result)
        if match:
            return match.group()
        else:
            return "No number found in response."
    except Exception as e:
        print(f"Error getting response from OpenAI: {e}")
        return "An error occurred while processing your query."

# Apply the prediction function to each row
def predict_missing_value(row):
    news_content = row['news']   # Full news content from the 'news' column
    headline = row['headline']   # Headline with a missing number
    return get_response(news_content, headline)

# Apply the function to your dataframe
data['predicted_value'] = data.apply(predict_missing_value, axis=1)

In [ ]:
data.to_csv('predicted_numbers.csv', index=False)